# DSCI 100 Project Group 1 
**By: Tima Komarov (82239450),**
<br>

In [16]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.



In [54]:
players <- read_csv("https://raw.githubusercontent.com/crmnwn06/minecraft-data/refs/heads/main/players.csv" )
sessions <- read_csv("https://raw.githubusercontent.com/crmnwn06/minecraft-data/refs/heads/main/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [55]:
set.seed(2025)
clean_player <- players |>
select(played_hours, Age, subscribe)
head(clean_player)

played_hours,Age,subscribe
<dbl>,<dbl>,<lgl>
30.3,9,TRUE
3.8,17,TRUE
0.0,17,FALSE
0.7,21,TRUE
0.1,21,TRUE
0.0,17,TRUE


In [53]:
set.seed(2025)

clean_player <- clean_player |>
mutate(subscribe = as_factor(subscribe)) 
(clean_player)


subscribe,played_hours,Age,na.rm
<fct>,<dbl>,<dbl>,<lgl>
TRUE,30.3,9,TRUE
TRUE,3.8,17,TRUE
FALSE,0.0,17,TRUE
TRUE,0.7,21,TRUE
TRUE,0.1,21,TRUE
TRUE,0.0,17,TRUE
TRUE,0.0,19,TRUE
FALSE,0.0,21,TRUE
TRUE,0.1,17,TRUE


In [50]:
#Pre modeling work

#splits
player_split <- initial_split(clean_player, prop = 0.8, strata = subscribe)
player_train <- training(player_split)
player_test <- testing(player_split)

#recipe work 
player_recipe <- recipe(subscribe ~ Age + played_hours, data = player_train) |>
  step_center(all_predictors()) |>
  step_scale(all_predictors())

#spec
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

#folds 
#Lmk numbers of folds you guys want to do just kept same as worksheet for now
player_fold <- vfold_cv(player_train, v = 5, strata = subscribe)


In [51]:
#tuning
k_vals <- tibble(neighbors = seq(from = 1, to = 5, by = 1))

knn_results <- workflow() |>
       add_recipe(player_recipe) |>
       add_model(knn_tune) |>
       tune_grid(resamples = player_fold, grid = k_vals) |>
       collect_metrics()
knn_results

→ A | error:   Assigned data `orig_rows` must be compatible with existing data.
               ✖ Existing data has 31 rows.
               ✖ Assigned data has 32 rows.
               ℹ Only vectors of size 1 are recycled.
               Caused by error in `vectbl_recycle_rhs_rows()`:
               ! Can't recycle input of size 32 to size 31.

There were issues with some computations   A: x1

→ B | error:   Assigned data `orig_rows` must be compatible with existing data.
               ✖ Existing data has 30 rows.
               ✖ Assigned data has 31 rows.
               ℹ Only vectors of size 1 are recycled.
               Caused by error in `vectbl_recycle_rhs_rows()`:
               ! Can't recycle input of size 31 to size 30.

There were issues with some computations   A: x1
There were issues with some computations   A: x1   B: x1





neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.4516129,3,0.01862420,Preprocessor1_Model1
1,roc_auc,binary,0.4809783,3,0.01849690,Preprocessor1_Model1
2,accuracy,binary,0.4623656,3,0.01075269,Preprocessor1_Model2
2,roc_auc,binary,0.4601449,3,0.06070868,Preprocessor1_Model2
3,accuracy,binary,0.5376344,3,0.05986843,Preprocessor1_Model3
3,roc_auc,binary,0.4791667,3,0.08279996,Preprocessor1_Model3
4,accuracy,binary,0.5161290,3,0.03225806,Preprocessor1_Model4
4,roc_auc,binary,0.5081522,3,0.06424777,Preprocessor1_Model4
5,accuracy,binary,0.5268817,3,0.04686988,Preprocessor1_Model5


# Introduction
(Intro Here)

# Methods and Results 
(Results)

# Discussion
(Discussion)

# References